# `marginal.py`

This notebook tests the `marginal` module.

This module contains the methods to marginalise FIGARO draws along the desired axis. The idea is to use the fact that marginalising a multivariate Gaussian distribution over a subspace is equivalent to remove the corresponding entries from mean and covariance of the multivariate Gaussian (e.g. [Robb J. Muirhead, Aspects of Multivariate Statistical Theory](http://www.ru.ac.bd/wp-content/uploads/sites/25/2019/03/401_06_Muirhead_Aspects-of-multivariate-statistical-theory.pdf), th. 1.2.7)

In [ ]:
from figaro.mixture import DPGMM
from scipy.stats import multivariate_normal as mn
from corner import corner
import numpy as np

mu    = np.zeros(2)
sigma = np.array([[2,1.5],[1.5,2]])

samples = mn(mu, sigma).rvs(1000)

c = corner(samples)

In [ ]:
from tqdm import tqdm

mix   = DPGMM([[-8,8],[-8,8]])
draws = np.array([mix.density_from_samples(samples) for _ in tqdm(range(100))])

## `_marginalise`

This method marginalises a single FIGARO draw by removing the corresponding entries from means and covariances of the DPGMM.

In [ ]:
from figaro.marginal import _marginalise
from figaro.utils import recursive_grid

d = draws[0]
grid, dgrid = recursive_grid(d.bounds, np.ones(2, dtype = int)*1000)

p = d.pdf(grid).reshape(1000,1000)

First dimension:

In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(d.bounds[0,0], d.bounds[0,1], 1002)[1:-1]

# Numerical marginalisation
p1 = p.sum(axis = 1)*dgrid[1]
# Analytical marginalisation
marg_d = _marginalise(d, 1)

plt.plot(x, p1)
plt.plot(x, marg_d.pdf(x), ls = '--')

np.allclose(p1, marg_d.pdf(x), atol = 1e-15)

Second dimension:# Numerical marginalisation
p1 = p.sum(axis = 1)*dgrid[1]
# Analytical marginalisation
marg_d = _marginalise(d, 1)

plt.plot(x, p1)
plt.plot(x, marg_d.pdf(x))

np.allclose(p1, marg_d.pdf(x), atol = 1e-15)

In [ ]:
# Numerical marginalisation
p2 = p.sum(axis = 0)*dgrid[0]
# Analytical marginalisation
marg_d = _marginalise(d, 0)

plt.plot(x, p2)
plt.plot(x, marg_d.pdf(x), ls = '--')

np.allclose(p2, marg_d.pdf(x), atol = 1e-15)

Marginalising on all dimensions raises an error:

In [ ]:
marg_d = _marginalise(d, [0,1])

## `marginalise`

This method is a wrapper for the private `_marginalise` that handles the possibility of having a list of draws. If the `axis` argument is an empty list, no marginalisation is done.

In [ ]:
from figaro.marginal import marginalise
from figaro.utils import plot_median_cr

d1 = marginalise(draws, 0)
plot_median_cr(d1, save = False, show = True)